# Placement of guests

Temos 3 cadeiras numa fila (*esquerda*, *meio*, *direita*), e precisamos de distribuir por elas 3 convidados (*Ana*, *Susana* e *Pedro*), com as seguintes restrições:

  * A Ana não quer ficar sentada à beira do Pedro.
  * A Ana não quer ficar na cadeira da esquerda.
  * A Susana não se quer sentar à esquerda do Pedro.

Será possível sentar os convidados? Como?

Começamos por instalar a biblioteca PySAT (https://pysathq.github.io/index.html).

In [1]:
!pip install python-sat[pblib,aiger]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.7 MB/s eta 0:00:00
  Created wheel for pypblib: filename=pypblib-0.0.4-cp310-cp310-linux_x86_64.whl size=2973439 sha256=dbdca678193973a1a3a4d212ecdd32a666736437a0f07eec6f661a222d83fe4c
  Stored in directory: /root/.cache/pip/wheels/a8/3d/9c/b980a09d6084eaeb9b336edbe20371c53c2550226daeee2025
Successfully built pypblib


Criamos uma variável proposicional para cada par (*pessoa*, *cadeira*)
e um dicionário, `X`, de forma a fazer o mapeamento entre cada variável proposicional  `X[p][c]`  e o valor inteiro que lhe corresponde no formato DIMACS CNF.

In [5]:
from pysat.solvers import Minisat22

ps = ["Ana","Susana","Pedro"]
cs = ["esquerda","meio","direita"]

P = len(ps)
C = len(cs)

y = 1
X = {}
for i in range(P):
    X[i] = {}
    for j in range(C):
        X[i][j] = y
        y += 1

s = Minisat22()

# Todas as pessoas estão sentadas.
for i in range(P):
    s.add_clause([X[i][j] for j in range(C)])

# No máximo uma pessoa por cadeira.
for c in range(C):
    for a in range(P-1):
        for b in range(a+1,P):
            s.add_clause([-X[a][c], -X[b][c]])


# A Ana não quer ficar sentada à beira do Pedro.
s.add_clause([-X[0][0], -X[2][1]])
s.add_clause([-X[0][2], -X[2][1]])
s.add_clause([-X[0][1], -X[2][0]])
s.add_clause([-X[0][1], -X[2][2]])

# A Ana não quer ficar na cadeira da esquerda.
s.add_clause([-X[0][0]])

# A Susana não se quer sentar à esquerda do Pedro.
s.add_clause([-X[1][1], -X[2][2]])
s.add_clause([-X[2][1], -X[1][0]])


if s.solve() == True:
    m = s.get_model()
    print(m)
    for p in range(P):
        for c in range(C):
                if m[X[p][c]-1]>0:
                    print("%s - %s senta-se na cadeira %s." % (X[p][c], ps[p],cs[c]))
else:
    print("Não é possível sentar os convidados com estas restrições.")





[-1, -2, 3, -4, 5, -6, 7, -8, -9]
3 - Ana senta-se na cadeira direita.
5 - Susana senta-se na cadeira meio.
7 - Pedro senta-se na cadeira esquerda.
